# Datenaufbereitung

In diesem Notebook stellen wir den aufgeräumten Datensatz her, der den Karten und Modellen im Notebook über Autos und Landschaft zugrundeliegt. Die hier angewandten Techniken beinhalten:

- Download und Auslesen von Excel-Tabellen
- Zusammenfügen von Daten aus unterschiedlichen Quellen
- Umgang mit georeferenzierten Raster, und Vektordaten
- weil's geht: Webcrawling von tabellarischen Daten aus der Wikipedia

In [1]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import rioxarray
from pyproj import CRS
import rpy2
%load_ext rpy2.ipython

In [ ]:
%%R
library(tidyverse)
library(readxl)
library(terra)
library(spatialEco)
library(raster)

# Download der öffentlichen Daten

- **Zulassungsdaten:**  
  Kraftfahrtbundesamt  
  https://www.kba.de/SharedDocs/Downloads/DE/Statistik/Fahrzeuge/FZ1/fz1_2022.xlsx

- **Landkreisgrenzen und Höhenmodell:**  
  Geodatenzentrum des Bundesamtes für Kartographie und Geodäsie  
  https://daten.gdz.bkg.bund.de/produkte/vg/vg2500/aktuell/vg2500_12-31.utm32s.shape.zip  
  https://daten.gdz.bkg.bund.de/produkte/dgm/dgm200/aktuell/dgm200.utm32s.geotiff.zip

- **Haushaltseinkommen pro Kopf:**  
  Statistische Ämter des Bundes und der Länder  
  https://www.statistikportal.de/sites/default/files/2022-10/vgrdl_r2b3_bs2021_0.xlsx
  

In [ ]:
%%R
dir.create("download", showWarnings = FALSE)
dir.create("data", showWarnings = FALSE)
options(timeout = 300) # Höhenmodell hatte timeouts beim Download

# Zulassungsdaten:
download.file("https://www.kba.de/SharedDocs/Downloads/DE/Statistik/Fahrzeuge/FZ1/fz1_2022.xlsx?__blob=publicationFile&v=5", destfile = "download/Zulassungen.xlsx")
download.file("https://daten.gdz.bkg.bund.de/produkte/vg/vg2500/aktuell/vg2500_12-31.utm32s.shape.zip", destfile = "download/Kreisgrenzen.zip")
download.file("https://daten.gdz.bkg.bund.de/produkte/dgm/dgm200/aktuell/dgm200.utm32s.geotiff.zip", destfile = "download/Höhenmodell.zip")
download.file("https://www.statistikportal.de/sites/default/files/2022-10/vgrdl_r2b3_bs2021_0.xlsx", destfile = "download/Einkommen.xlsx")

unzip("download/Kreisgrenzen.zip", exdir = "download/Kreisgrenzen")
unzip("download/Höhenmodell.zip", exdir = "download/Höhenmodell")

# Zulassungsdaten

Gesucht ist je Kreis die Anzahl der PKW-Zulassungen, unterschieden nach Hubraum und nach Euro-Abgasnorm.
Besonders der Hubraum dient uns als Anhaltspunkt, da wir davon ausgehen, dass abgesehen von der Hügeligkeit der umgebenden Landschaft keine anderen, von Kreis zu Kreis systematisch variierenden Faktoren existieren, die vorhersagen würden, dass hier stärkere PKWs als dort notwendig wären.

In [6]:
%%R
pkw_cm3 <- read_excel("download/Zulassungen.xlsx",
                      sheet = "FZ1.1",
                      skip = 8,
                      .name_repair = ~ vctrs::vec_as_names(..., repair = "unique", quiet = TRUE))

colnames(pkw_cm3) <- str_replace_all(colnames(pkw_cm3), "[^a-zA-Z0-9]", "_")

pkw_cm3 <- pkw_cm3 |> 
  transmute( id_bezirk = Statistische_Kennziffer_und_Zulassungsbezirk,
             krs_code = str_extract(id_bezirk, "^[0-9]+"),
             id_bezirk = NULL,
             pkw_gesamt = insgesamt___9,
             max_cm3_1399 = Hubraum__bis___1_399_cm_,
             max_cm3_1999 = `1_400__bis__1_999_cm_`,
             cm3_over_1999 = `2_000__und__mehr_cm_` ) |> 
  filter(!is.na(krs_code))



pkw_euro <- read_excel("download/Zulassungen.xlsx",
                       sheet = "FZ1.2",
                       skip = 8,
                       .name_repair = ~ vctrs::vec_as_names(..., repair = "unique", quiet = TRUE))

colnames(pkw_euro) <- str_replace_all(colnames(pkw_euro), "[^a-zA-Z0-9]", "_")

pkw_euro <- pkw_euro |> 
  transmute( krs_code = str_extract(`___3`, "^[0-9]+"),
             euro_1 = Euro_1___12,
             euro_2 = Euro_2___13,
             euro_3 = Euro_3___14,
             euro_4 = Euro_4___15,
             euro_5 = Euro_5___16,
             euro_6 = Euro_6___17 ) |> 
  filter(!is.na(krs_code))

Wir erstellen den Gesamtdatensatz `data` und fügen als Erstes die Zulassungsdaten hinzu:

In [16]:
%%R
print(head(pkw_cm3))

data <- pkw_cm3 |> 
  inner_join(pkw_euro, by = c("krs_code")) |> 

  # zusätzlich zu Absolutzahlen die Anteile am Gesamtbestand:
  mutate( max_cm3_1399_pct = max_cm3_1399 / pkw_gesamt * 100,
          max_cm3_1999_pct = max_cm3_1999 / pkw_gesamt * 100,
          cm3_over_1999_pct = cm3_over_1999 / pkw_gesamt * 100,
          euro_1_pct = euro_1 / pkw_gesamt * 100,
          euro_2_pct = euro_2 / pkw_gesamt * 100,
          euro_3_pct = euro_3 / pkw_gesamt * 100,
          euro_4_pct = euro_4 / pkw_gesamt * 100,
          euro_5_pct = euro_5 / pkw_gesamt * 100,
          euro_6_pct = euro_6 / pkw_gesamt * 100
          )

# A tibble: 6 × 5
  krs_code pkw_gesamt max_cm3_1399 max_cm3_1999 cm3_over_1999
  <chr>         <dbl>        <dbl>        <dbl>         <dbl>
1 08111        302792        97105       143689         53363
2 08115        257554        85378       119854         45718
3 08116        337566       120022       159225         52259
4 08117        168923        61381        79315         25905
5 08118        336361       119100       159926         52017
6 08119        269889        96089       126480         43112


# Haushaltseinkommen

Für jeden Kreis bestehen Daten über das durchschnittliche verfügbare Haushaltseinkommen pro Kopf. Wir ordnen diesen Wert den Kreisen in unserem Datensatz zu:

In [23]:
%%R
income <- read_excel("download/Einkommen.xlsx",
                     sheet = "2.4",
                     range = "A5:AH451" ) |>
  slice(-1) |> 
  transmute( krs_code = `Regional-schlüssel`,
             nuts3 = `NUTS 3`,
             income = `2020`) |> 
  # Länder & Reg.-Bez. ignorieren:
  filter(nuts3 == 3) |> 
  # die 2-stelligen Codes von HH und B mit Nullen auffüllen
  # (mit Dank an dieses krumme Code-Format, das Nullen an Anfang und Ende erlaubt
  # und dann auch noch inkonsistent benutzt wird)
  mutate(krs_code = krs_code |> map_chr(~ paste0(., paste0(rep("0", 5-nchar(.)), collapse = "")))) |> 
  dplyr::select(-nuts3)

print(head(income))
    
data <- data |> 
  inner_join( income, by = "krs_code" )

column_names <- colnames(data)

# A tibble: 6 × 2
  krs_code income
  <chr>     <dbl>
1 08111     26503
2 08115     26608
3 08116     26577
4 08117     24810
5 08118     27079
6 08119     26166


# Umrisse der (Land-)Kreise

Um zu erfahren, wie rauh das Gelände durchschnittlich in jedem Kreis ist, brauchen wir seinen Umriss, um damit den betreffenden Teil aus dem deutschlandweiten Höhenraster "auszustechen" und in ihm den Durchschnittswert zu berechnen. Außerdem benutzen wir diese Umrissdaten gleich für Kartendarstellungen mit.

In [13]:
kreise = gpd.read_file("download/Kreisgrenzen/vg2500_12-31.utm32s.shape/vg2500/VG2500_KRS.shp").\
           loc[:,["AGS", "GEN", "geometry"]].\
           rename({"AGS": "krs_code",
                   "GEN": "name"}, axis = 1)
kreise.set_index("krs_code", inplace = True)
kreise = kreise.to_crs(epsg = "4326")

In [17]:
kreise.head()

,name,geometry
krs_code,,
01001,Flensburg,"POLYGON ((9.48982 54.82296, 9.50467 54.81004, ..."
01002,Kiel,"POLYGON ((10.19057 54.38909, 10.18370 54.38910..."
01003,Lübeck,"POLYGON ((10.89297 53.95695, 10.90337 53.95705..."
01004,Neumünster,"POLYGON ((10.03454 54.12915, 10.02277 54.10938..."
01051,Dithmarschen,"MULTIPOLYGON (((8.68760 54.05944, 8.69258 54.0..."


In [47]:
import rpy2.robjects as robjects

# Datensatz aus R überführen und Datentypen korrigieren:
data = pd.DataFrame(robjects.globalenv["data"]).transpose()
data.columns = robjects.globalenv["column_names"]
data.set_index("krs_code", inplace = True)
data = data.astype(dict( pkw_gesamt=int, max_cm3_1399=int, max_cm3_1999=int, cm3_over_1999=int,
                  euro_1=int, euro_2=int, euro_3=int, euro_4=int, euro_5=int, euro_6=int,
                  max_cm3_1399_pct=float, max_cm3_1999_pct=float, cm3_over_1999_pct=float,
                  euro_1_pct=float, euro_2_pct=float, euro_3_pct=float, euro_4_pct=float,
                  euro_5_pct=float, euro_6_pct=float, income=int) )

# mit Geodaten zusammenführen:
data = pd.merge(left  = data,   left_on  = "krs_code",
                right = kreise, right_on = "krs_code")
data = gpd.GeoDataFrame(data)

data.head()

,pkw_gesamt,max_cm3_1399,max_cm3_1999,cm3_over_1999,euro_1,euro_2,euro_3,euro_4,euro_5,euro_6,...,cm3_over_1999_pct,euro_1_pct,euro_2_pct,euro_3_pct,euro_4_pct,euro_5_pct,euro_6_pct,income,name,geometry
krs_code,,,,,,,,,,,,,,,,,,,,,
08111,302792,97105,143689,53363,3156,12642,11901,50920,51020,157147,...,17.623649,1.042300,4.175143,3.930421,16.816825,16.849851,51.899324,26503,Stuttgart,"POLYGON ((9.24427 48.84680, 9.23586 48.84218, ..."
08115,257554,85378,119854,45718,2080,9605,10798,46389,52156,125054,...,17.750841,0.807598,3.729315,4.192519,18.011368,20.250511,48.554478,26608,Böblingen,"POLYGON ((9.06688 48.75586, 9.04633 48.74718, ..."
08116,337566,120022,159225,52259,3445,14483,16288,67556,73253,148424,...,15.481121,1.020541,4.290420,4.825130,20.012679,21.700349,43.968883,26577,Esslingen,"POLYGON ((9.40979 48.77320, 9.41317 48.76437, ..."
08117,168923,61381,79315,25905,1451,7024,8481,35226,40118,71031,...,15.335389,0.858971,4.158108,5.020631,20.853288,23.749282,42.049336,24810,Göppingen,"POLYGON ((9.62622 48.77426, 9.63474 48.77244, ..."
08118,336361,119100,159926,52017,3211,15535,16762,69988,73220,145927,...,15.464635,0.954629,4.618550,4.983336,20.807406,21.768279,43.384043,27079,Ludwigsburg,"MULTIPOLYGON (((9.41678 49.04794, 9.39967 49.0..."


# Höhenmodell für die Fläche Deutschlands

Mit dem *Terrain Ruggedness Index (TRI)* ([Riley et al., 1999](#Bibliografie)) existiert ein Maß für die Rauhheit oder Hügeligkeit einer Landschaft. Es basiert auf einem Höhenmodell, also einem Raster, das für jeden Punkt in einer gewissen Auflösung die Höhe ü. M. angibt. Wir laden das Höhenmodell in R und erstellen daraus ein Raster, das den TRI repräsentiert:

In [35]:
%%R
dem_d <- terra::rast("download/Höhenmodell/dgm200.utm32s.geotiff/dgm200/dgm200_utm32s.tif")
ruggedness_d <- spatialEco::tri(dem_d) # dieser Prozess dauert wenige Minuten
terra::writeRaster(ruggedness_d, "data/TRI_DEM200_Deutschland.tif", overwrite = TRUE)

In Python laden wir das Raster und berechnen je Kreis einen Durchschnittswert:

In [48]:
ruggedness_d = rioxarray.open_rasterio("data/TRI_DEM200_Deutschland.tif")
ruggedness_d = ruggedness_d.rio.set_crs(25832)
ruggedness_d = ruggedness_d.rio.reproject("EPSG:4326")

rug_averages = []

for i in range(len(data)):
  # nimm jeden Kreis-Umriss,
  this_geo = [data.iloc[i]["geometry"]]
  # isoliere den Teil des TRI-Rasters, der darunter liegt
  this_raster = ruggedness_d.rio.clip(this_geo)
  # und ermittle durchschnittliche Rauhheit
  this_avg = float(this_raster.mean())
  rug_averages.append(this_avg)

data["m_rugged"] = rug_averages

data.head()

,pkw_gesamt,max_cm3_1399,max_cm3_1999,cm3_over_1999,euro_1,euro_2,euro_3,euro_4,euro_5,euro_6,...,euro_1_pct,euro_2_pct,euro_3_pct,euro_4_pct,euro_5_pct,euro_6_pct,income,name,geometry,m_rugged
krs_code,,,,,,,,,,,,,,,,,,,,,
08111,302792,97105,143689,53363,3156,12642,11901,50920,51020,157147,...,1.042300,4.175143,3.930421,16.816825,16.849851,51.899324,26503,Stuttgart,"POLYGON ((9.24427 48.84680, 9.23586 48.84218, ...",42.795776
08115,257554,85378,119854,45718,2080,9605,10798,46389,52156,125054,...,0.807598,3.729315,4.192519,18.011368,20.250511,48.554478,26608,Böblingen,"POLYGON ((9.06688 48.75586, 9.04633 48.74718, ...",34.500069
08116,337566,120022,159225,52259,3445,14483,16288,67556,73253,148424,...,1.020541,4.290420,4.825130,20.012679,21.700349,43.968883,26577,Esslingen,"POLYGON ((9.40979 48.77320, 9.41317 48.76437, ...",52.653061
08117,168923,61381,79315,25905,1451,7024,8481,35226,40118,71031,...,0.858971,4.158108,5.020631,20.853288,23.749282,42.049336,24810,Göppingen,"POLYGON ((9.62622 48.77426, 9.63474 48.77244, ...",65.124016
08118,336361,119100,159926,52017,3211,15535,16762,69988,73220,145927,...,0.954629,4.618550,4.983336,20.807406,21.768279,43.384043,27079,Ludwigsburg,"MULTIPOLYGON (((9.41678 49.04794, 9.39967 49.0...",37.324421


In [53]:
data = data.drop("geometry", axis = 1)

import csv
data.to_csv("data/krs_pkw_rug.csv",
            index = True,
            quoting = csv.QUOTE_NONNUMERIC)

# Bibliografie

Zum Nachschlagen der Literaturverweise auf wissenschaftliche Arbeiten:

Riley, S. J., DeGloria, S. D., & Elliot, R. (1999). [Index that quantifies topographic heterogeneity.](http://download.osgeo.org/qgis/doc/reference-docs/Terrain_Ruggedness_Index.pdf) *Intermountain Journal of Sciences*, 5(1-4), 23-27.